In [1]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
import cv2
import numpy as np
import glob
from skimage import io

width=200
height=200


Data=[]
Target=[]
Test=[]
Target1=[]



# genuies signature
file_list = glob.glob(r'/content/drive/My Drive/DataSet/Signature/dataset/genuines/*.png')
for i in file_list:
  im=cv2.imread(i)
  im=cv2.resize(im,(width,height))
  Data.append(im)
  Target.append(0)


  # forgeries signature
file_list1 = glob.glob('/content/drive/My Drive/DataSet/Signature/dataset/forgeries/*.png')

for i in file_list:
  im=cv2.imread(i)
  im=cv2.resize(im,(width,height))
  Data.append(im)
  Target.append(1)

# forgeries signature Test
file_list1 = glob.glob(r'/content/drive/My Drive/DataSet/Signature/sample_Signature/forged/*.PNG')

for i in file_list1:
  im=cv2.imread(i)
  im=cv2.resize(im,(width,height))
  Test.append(im)
  Target1.append(0)

file_list = glob.glob(r'/content/drive/My Drive/DataSet/Signature/sample_Signature/genuine/*.PNG')
# genuies signature
for i in file_list:
  im=cv2.imread(i)
  im=cv2.resize(im,(width,height))
  Test.append(im)
  Target1.append(1)

# genuies signature
file_list1 = glob.glob(r'/content/drive/My Drive/DataSet/Signature/sample_Signature/genuine/*.png')
for i in file_list1:
  im=cv2.imread(i)
  im=cv2.resize(im,(width,height))
  Test.append(im)
  Target1.append(0)

  # forgeries signature
file_list1 = glob.glob(r'/content/drive/My Drive/DataSet/Signature/sample_Signature/forged/*.png')

for i in file_list1:
  im=cv2.imread(i)
  im=cv2.resize(im,(width,height))
  Test.append(im)
  Target1.append(1)



In [ ]:
from tensorflow.keras.utils import to_categorical
y_train = to_categorical(Target)

In [ ]:
from tensorflow.keras.utils import to_categorical
y_test = to_categorical(Target1)

In [ ]:
Final_Data=np.reshape(Data,(len(Data),height,width,3))

In [ ]:
Final_Test=np.reshape(Test,(len(Test),height,width,3))

In [ ]:
from keras.layers import Input, Lambda, Dense, Flatten
from keras.models import Model
from keras.applications.vgg16 import VGG16
from keras.applications.vgg16 import preprocess_input
import numpy as np
from keras import layers
import keras

In [ ]:
vgg=VGG16(include_top=False,weights='imagenet',input_shape=(200,200,3))

58889256/58889256 [==============================] - 0s 0us/step


In [ ]:
for layer in vgg.layers:
    layer.trainable =False

In [ ]:
# CNN LSTM
from keras import layers
from keras.layers import Input
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Dropout
from keras.layers.convolutional import Conv2D,SeparableConv2D
from keras.layers.convolutional import MaxPooling2D
from keras.layers import TimeDistributed
from keras.layers import LSTM
from keras.models import Model

conv1=SeparableConv2D(64,3, activation='relu',padding='same')(vgg.output)
max1=MaxPooling2D(2)(conv1)
conv2=SeparableConv2D(64,3, activation='relu',padding='same')(max1)
max2=MaxPooling2D(2)(conv2)
Time1=layers.TimeDistributed(Flatten())(max2)
lstm1=LSTM(150)(Time1)
des1 =Dense(50, activation='relu')(lstm1)
Extract_Model = Model(vgg.input, des1)
des2 =Dense(20, activation='relu')(des1)
out_layer =Dense(2, activation='softmax')(des2)
model = Model(vgg.input, out_layer)
model.summary()

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

verbose, epochs, batch_size = 1, 10, 32
model.fit(Final_Data,y_train, epochs=epochs, batch_size=batch_size, verbose=verbose)

Model: "model_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 200, 200, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 200, 200, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 200, 200, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 100, 100, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 100, 100, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 100, 100, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 50, 50, 128)       0   

In [ ]:
trainFeature=(Extract_Model.predict(Final_Data))
testFeature=(Extract_Model.predict(Final_Test))

10/10 [==============================] - 102s 10s/step


In [ ]:
from sklearn import svm
classifier_svm = svm.LinearSVC()
classifier_svm.fit(trainFeature,Target)

LinearSVC()

In [ ]:
from sklearn import tree
from sklearn.metrics import classification_report, accuracy_score
classifier_tree = tree.DecisionTreeClassifier()
classifier_tree.fit(trainFeature,Target)



DecisionTreeClassifier()

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
classifier_Knn = KNeighborsClassifier(n_neighbors=3)
classifier_Knn.fit(trainFeature,Target)

KNeighborsClassifier(n_neighbors=3)

In [ ]:
from sklearn.ensemble import  VotingClassifier
classifier_Voting = VotingClassifier(estimators=[('SVM', classifier_svm), ('TREE', classifier_tree), ('KNN', classifier_Knn)],weights=[1,1,1])
classifier_Voting.fit(trainFeature,Target)
Class_Voting=classifier_Voting.predict(testFeature)

In [ ]:
from sklearn.metrics import classification_report, accuracy_score
print(classification_report(Target1,Class_Voting))

              precision    recall  f1-score   support

           0       0.60      0.91      0.72       180
           1       0.43      0.11      0.17       120

    accuracy                           0.59       300
   macro avg       0.52      0.51      0.45       300
weighted avg       0.54      0.59      0.50       300



In [ ]:
print('Accuracy= ' + str(round(accuracy_score(Target1, Class_Voting)*100)))

Accuracy= 59
